In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import joblib

In [2]:
final_rmse = 5000

In [3]:
def download_data(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date)
    return df

In [4]:
def prepare_data(df, sequence_length):
    data = df.filter(['Close'])
    df = df.dropna()
    df = df[~df.index.duplicated(keep='last')]
    df = df.values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []

    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    split_index = int(len(scaled_data) * 0.8)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    
    return X_train, X_test, y_train, y_test, scaler

In [5]:
def build_gru_model(X_train, units=50, layers=2, activation='tanh', optimizer='adam'):
    model = Sequential()
    model.add(GRU(units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    for _ in range(layers - 1):
        model.add(GRU(units, return_sequences=True))
    model.add(GRU(units*2))
    model.add(Dense(25, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [6]:
def train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers):
    
    global final_rmse
    
    loss_history = keras.callbacks.History()
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'model_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.h5', monitor='val_loss', save_best_only=True)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping, model_checkpoint, loss_history])

    gru_loss_history = loss_history.history['loss']
    
    gru_predictions = model.predict(X_test)
    gru_predictions = scaler.inverse_transform(gru_predictions)
    
    y_test = y_test.reshape(-1,1)
    y_test = scaler.inverse_transform(y_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, gru_predictions))
    print(f"Root Mean Squared Error (Testing Dataset): {rmse}")
    
    if rmse <= final_rmse:
        final_rmse = rmse
        joblib.dump(model, "LTC_model_gru.pkl")
        plt.plot(y_test, label='Actual')
        plt.plot(gru_predictions, label='Predicted')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.legend()
        plt.savefig(f'Plot_actualvspredicted/LTC_actual_vs_predicted_GRU.png')
        plt.close()
    
        plt.figure(figsize=(10, 6))
        plt.plot(loss_history.history['loss'], label='Training Loss')
        plt.plot(loss_history.history['val_loss'], label='Validation Loss')
        plt.title('Epoch Loss Curve')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig(f'loss_curve/LTC_loss_curve_GRU.png')
        plt.close()
    
    return model, rmse


In [7]:
symbol = 'LTC-USD'
start_date = '2020-01-01'
end_date = '2024-01-31'

In [8]:
epochs_range = [100]
batch_sizes = [4,8]
sequence_lengths = [25,20]
units_range = [50, 100]
layers_range = [2, 3]

In [9]:
results = []

In [10]:
df = download_data(symbol, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [11]:
for epochs in epochs_range:
    for batch_size in batch_sizes:
        for sequence_length in sequence_lengths:
            for units in units_range:
                for layers in layers_range:
                    print(f"Training model with epochs={epochs}, batch_size={batch_size}, sequence_length={sequence_length}, units={units}, layers={layers}")
                    X_train, X_test, y_train, y_test, scaler = prepare_data(df, sequence_length)
                    model = build_gru_model(X_train, units=units, layers=layers)
                    model, rmse = train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers)
                    results.append({'epochs': epochs, 'batch_size': batch_size, 'sequence_length': sequence_length, 'units': units, 'layers': layers, 'rmse': rmse})

Training model with epochs=100, batch_size=4, sequence_length=25, units=50, layers=2


Epoch 1/100

268/268 [==============================] - 21s 41ms/step - loss: 0.0033 - val_loss: 8.6574e-04
Epoch 2/100
  3/268 [..............................] - ETA: 7s - loss: 9.0113e-04

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 7s 27ms/step - loss: 0.0014 - val_loss: 1.9361e-04
Epoch 3/100
268/268 [==============================] - 7s 27ms/step - loss: 0.0010 - val_loss: 4.3030e-04
Epoch 4/100
268/268 [==============================] - 7s 27ms/step - loss: 0.0011 - val_loss: 4.4641e-04
Epoch 5/100
268/268 [==============================] - 7s 27ms/step - loss: 9.8405e-04 - val_loss: 4.8907e-04
Epoch 6/100
268/268 [==============================] - 8s 30ms/step - loss: 8.3237e-04 - val_loss: 2.3966e-04
Epoch 7/100
268/268 [==============================] - 9s 32ms/step - loss: 7.5033e-04 - val_loss: 2.5283e-04
Epoch 8/100
268/268 [==============================] - 8s 30ms/step - loss: 0.0011 - val_loss: 8.0665e-04
Epoch 9/100
268/268 [==============================] - 8s 28ms/step - loss: 0.0010 - val_loss: 8.5369e-05
Epoch 10/100
268/268 [==============================] - 8s 32ms/step - loss: 8.6813e-04 - val_loss: 1.7390e-04
Epoch 11/100
268/268 [===================

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 10s 39ms/step - loss: 0.0013 - val_loss: 2.1000e-04
Epoch 3/100
268/268 [==============================] - 11s 40ms/step - loss: 0.0014 - val_loss: 0.0010
Epoch 4/100
268/268 [==============================] - 10s 37ms/step - loss: 0.0010 - val_loss: 1.3786e-04
Epoch 5/100
268/268 [==============================] - 10s 36ms/step - loss: 9.5864e-04 - val_loss: 1.5215e-04
Epoch 6/100
268/268 [==============================] - 11s 40ms/step - loss: 0.0015 - val_loss: 1.1011e-04
Epoch 7/100
268/268 [==============================] - 10s 37ms/step - loss: 0.0011 - val_loss: 1.0230e-04
Epoch 8/100
268/268 [==============================] - 10s 37ms/step - loss: 0.0011 - val_loss: 1.1065e-04
Epoch 9/100
268/268 [==============================] - 10s 37ms/step - loss: 6.9604e-04 - val_loss: 6.6927e-04
Epoch 10/100
268/268 [==============================] - 10s 38ms/step - loss: 0.0012 - val_loss: 9.3678e-05
Epoch 11/100
268/268 [======================

...

134/134 [==============================] - 5s 35ms/step - loss: 0.0018 - val_loss: 6.2372e-04
Epoch 3/100
134/134 [==============================] - 5s 35ms/step - loss: 0.0014 - val_loss: 3.6362e-04
Epoch 4/100
134/134 [==============================] - 5s 35ms/step - loss: 0.0011 - val_loss: 2.1444e-04
Epoch 5/100
134/134 [==============================] - 5s 34ms/step - loss: 0.0010 - val_loss: 2.9073e-04
Epoch 6/100
134/134 [==============================] - 5s 34ms/step - loss: 9.8925e-04 - val_loss: 2.9186e-04
Epoch 7/100
134/134 [==============================] - 5s 36ms/step - loss: 8.8185e-04 - val_loss: 1.0177e-04
Epoch 8/100
134/134 [==============================] - 5s 34ms/step - loss: 7.5844e-04 - val_loss: 1.4384e-04
Epoch 9/100
134/134 [==============================] - 5s 36ms/step - loss: 7.1836e-04 - val_loss: 9.1062e-05
Epoch 10/100
134/134 [==============================] - 5s 35ms/step - loss: 9.0228e-04 - val_loss: 9.7864e-05
Epoch 11/100
134/134 [===============

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 5s 41ms/step - loss: 0.0012 - val_loss: 2.6202e-04
Epoch 3/100
134/134 [==============================] - 5s 37ms/step - loss: 0.0012 - val_loss: 1.0721e-04
Epoch 4/100
134/134 [==============================] - 6s 46ms/step - loss: 9.5027e-04 - val_loss: 1.0015e-04
Epoch 5/100
134/134 [==============================] - 5s 38ms/step - loss: 9.2314e-04 - val_loss: 1.2384e-04
Epoch 6/100
134/134 [==============================] - 5s 40ms/step - loss: 9.1684e-04 - val_loss: 1.7871e-04
Epoch 7/100
134/134 [==============================] - 5s 39ms/step - loss: 8.6949e-04 - val_loss: 1.4319e-04
Epoch 8/100
134/134 [==============================] - 5s 41ms/step - loss: 8.8225e-04 - val_loss: 3.8282e-04
Epoch 9/100
134/134 [==============================] - 5s 41ms/step - loss: 9.0560e-04 - val_loss: 3.8654e-04
Epoch 10/100
134/134 [==============================] - 5s 38ms/step - loss: 9.5607e-04 - val_loss: 9.0449e-05
Epoch 11/100
134/134 [=======

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 6s 48ms/step - loss: 0.0016 - val_loss: 1.2504e-04
Epoch 3/100
134/134 [==============================] - 6s 47ms/step - loss: 0.0010 - val_loss: 1.9746e-04
Epoch 4/100
134/134 [==============================] - 6s 47ms/step - loss: 8.5000e-04 - val_loss: 1.9459e-04
Epoch 5/100
134/134 [==============================] - 6s 46ms/step - loss: 8.4919e-04 - val_loss: 5.2936e-04
Epoch 6/100
134/134 [==============================] - 7s 49ms/step - loss: 0.0013 - val_loss: 9.1294e-05
Epoch 7/100
134/134 [==============================] - 7s 51ms/step - loss: 8.7736e-04 - val_loss: 2.3807e-04
Epoch 8/100
134/134 [==============================] - 6s 47ms/step - loss: 8.5120e-04 - val_loss: 2.2226e-04
Epoch 9/100
134/134 [==============================] - 7s 50ms/step - loss: 9.6116e-04 - val_loss: 9.9152e-04
Epoch 10/100
134/134 [==============================] - 7s 50ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 11/100
134/134 [===================

In [12]:
for result in results:
    print(result)

{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 3.0307075949892965}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 3, 'rmse': 2.818780594236671}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 2, 'rmse': 3.9352150870364486}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 3, 'rmse': 2.7356776955954634}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 50, 'layers': 2, 'rmse': 3.0711205664919556}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 50, 'layers': 3, 'rmse': 2.9960548053094502}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 100, 'layers': 2, 'rmse': 3.1642205867231112}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 100, 'layers': 3, 'rmse': 3.294834029586587}
{'epochs': 100, 'batch_size': 8, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 3.4574883363091344}
{'epochs